This exercise will require you to pull some data from the Quandl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
txt = open('key.txt', 'r')
API_KEY = txt.read()
txt.close()

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import collections

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
#limit=1 sends the latest row
url = f'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?limit=1&api_key={API_KEY}'
r = requests.get(url)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(r.text)

{"dataset_data":{"limit":1,"transform":null,"column_index":null,"column_names":["Date","Open","High","Low","Close","Change","Traded Volume","Turnover","Last Price of the Day","Daily Traded Units","Daily Turnover"],"start_date":"2000-06-07","end_date":"2020-12-01","frequency":"daily","data":[["2020-12-01",112.2,112.2,111.5,112.0,null,51.0,5703.0,null,null,null]],"collapse":null,"order":null}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
#1. Call Quandl API and collect data from AFX_X for the whole year 2017
afx_2017_url = f'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key={API_KEY}'
r_2017 = requests.get(afx_2017_url)

In [6]:
#2. Convert the returns JSON object into a Python dictionary
json_data = r_2017.json()
afx_dict = {}

for key, value in json_data.items():
    afx_dict[key] = value

In [7]:
#def get_index(colname):
#    column_names.index(colname)

def unpack_dict_values_needed(data_dict, colname):
    '''unpacks the desired column
    and returns a list of values'''
    #get dictionary data
    data = data_dict['dataset_data']['data']
    #get column names from dataset
    column_names = data_dict['dataset_data']['column_names']
    #define a list for values needed
    a_list = []
    
    #unpack data
    for d in data:
        a_list.append(d[column_names.index(colname)])
    
    #filters NoneType values
    a_list = [i for i in a_list if i]
    
    return a_list
    

In [8]:
#3. Calculate what the highest and lowest opening prices were for the stock in this period.
price = unpack_dict_values_needed(afx_dict, 'Open')
#maximum opening price for 2017
max(price)

53.11

In [9]:
#minimum opening price for 2017
min(price)

34.0

In [10]:
#4. What was the largest change in any one day (based on High and Low price)?
#get dictionary data
data = afx_dict['dataset_data']['data']
#get column names from dataset
column_names = afx_dict['dataset_data']['column_names']
delta_highlow=[]
for d in data:
    change=d[column_names.index('High')]-d[column_names.index('Low')]
    delta_highlow.append(change)
round(max(delta_highlow),2)

2.81

In [11]:
#5. What was the largest change between any two days (based on Closing Price)?
closing = unpack_dict_values_needed(afx_dict, 'Close' )
delta_closing = []
for x, y in zip(closing[1:], closing[:-1]):
    delta_closing.append(abs(x-y))
    
round(max(delta_closing),2)

2.56

In [12]:
#6. What was the average daily trading volume during this year?
volume = unpack_dict_values_needed(afx_dict, 'Traded Volume')
sum(volume)/len(volume)

89124.33725490196

In [13]:
#7. (Optional) What was the median trading volume during this year.
#   (Note: you may need to implement your own function for calculating the median.)
def median(data_list):
    '''returns the median'''
    sorted_list = sorted(data_list)
    if len(sorted_list) % 2:
        index=len(sorted_list)/2 - 0.5
        return sorted_list[int(index)]
    else:
        index=len(sorted_list)/2
        return (sorted_list[int(index-.5)]+sorted_list[int(index+.5)])/2

median(volume)

76286.0